In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-employee-access-challenge/sampleSubmission.csv
/kaggle/input/amazon-employee-access-challenge/test.csv
/kaggle/input/amazon-employee-access-challenge/train.csv


In [12]:
train_data = pd.read_csv('../input/amazon-employee-access-challenge/train.csv')

In [13]:
train_data.shape

(32769, 10)

In [ ]:
train_data.head()

In [14]:
train_data.dropna(axis=0, subset=['ACTION'], inplace=True)
y = train_data.ACTION
train_data.drop(['ACTION'], axis=1, inplace=True)

In [15]:
train_data.shape

(32769, 9)

In [16]:
categorical_cols = [cname for cname in train_data.columns if
                    train_data[cname].nunique() < 10 and 
                    train_data[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in train_data.columns if 
                train_data[cname].dtype in ['int64', 'float64']]

In [17]:
my_cols = categorical_cols + numerical_cols
reduced_train = train_data[my_cols].copy()

In [18]:
reduced_train.shape

(32769, 9)

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [20]:
numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


In [21]:
model = RandomForestRegressor(n_estimators=100, random_state=0)

In [22]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [23]:
clf.fit(reduced_train, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['RESOURCE', 'MGR_ID',
                                                   'ROLE_ROLLUP_1',
                                                   'ROLE_ROLLUP_2',
                                                   'ROLE_DEPTNAME',
                                                   'ROLE_TITLE',
                                                   'ROLE_FAMILY_DESC',
                                                   'ROLE_FAMILY',
                                                   'ROLE_CODE']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                   

In [24]:
test_data=pd.read_csv('../input/amazon-employee-access-challenge/test.csv',index_col='id')

In [25]:
test_data.shape

(58921, 9)

In [27]:
reduced_test=test_data[my_cols].copy()

In [29]:
reduced_test.shape

(58921, 9)

In [32]:
reduced_test.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
id,,,,,,,,,
1,78766,72734,118079,118080,117878,117879,118177,19721,117880
2,40644,4378,117961,118327,118507,118863,122008,118398,118865
3,75443,2395,117961,118300,119488,118172,301534,249618,118175
4,43219,19986,117961,118225,118403,120773,136187,118960,120774
5,42093,50015,117961,118343,119598,118422,300136,118424,118425


In [33]:
preds=model.predict(reduced_test)

In [34]:
print(preds[0:5])

[0.99 0.98 1.   0.99 1.  ]


In [37]:
preds_new=[]
for val in preds:
    if val>0.5:
        preds_new.append(1)
    else:
        preds_new.append(0)

In [44]:
print(1.5 in preds_new)

False


In [45]:
output = pd.DataFrame({'Id': reduced_test.index,
                       'Action': preds_new})
#output.to_csv('submission.csv', index=False)

In [46]:
output.shape

(58921, 2)

In [47]:
output.head()

,Id,Action
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [48]:
output.to_csv('sub5.csv', index=False)